#### Define the used fields from the UK biobank, and extract the patients diagnosed with vascular dementia

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from DataFields import DataFields
from DataFields import DateReportedFields
from ProjectFunctions import get_columns_from_chunk

UKBB_PATH = "~/biobank/ukb672220.csv"

# A dictionary containing the used features
datafields = DataFields

# Features of conditions specified by date
datereportedfields = DateReportedFields

In [2]:
# Filter by rows to extract patients with vascular dementia
def filter_by_vascular_dementia(chunk):
    col_1_filter = "Vascular Dementia Date First Reported"
    col_2_filter = "Date of vascular dementia report"
    col_combined = "Vascular Dementia Report Date"
    Y_col = "Has Vascular Dementia"
    
    # Filter rows where either col_1_filter or col_2_filter is not NaN
    filtered_chunk = chunk[chunk[col_1_filter].notna() | chunk[col_2_filter].notna()].copy()  # Make a copy here
    
    # Combine dates and add the new columns
    filtered_chunk.loc[:, col_combined] = pd.to_datetime(
        filtered_chunk[col_1_filter].combine_first(filtered_chunk[col_2_filter]), errors="coerce"
    )
    
    # Filter by dates to remove unspecified entries
    filtered_chunk.loc[:, Y_col] = (
        filtered_chunk[col_combined]
        .between(pd.Timestamp("1950-01-01"), pd.Timestamp("2030-12-31"))
        .astype(int)
    )

    return filtered_chunk


### Extract rows diagnosed with vascular dementia through the use of two fields

In [3]:
chunk_size = 10000

# Create an empty DataFrame to hold the filtered data
result_df = pd.DataFrame()

for chunk in pd.read_csv(UKBB_PATH, chunksize=chunk_size, low_memory=False):

    filtered_chunk = get_columns_from_chunk(chunk, datafields, oldest=True)

    filtered_chunk = filter_by_vascular_dementia(filtered_chunk)
    
    # Append to the result DataFrame
    result_df = pd.concat([result_df, filtered_chunk], ignore_index=True)

col_1_filter = "Vascular Dementia Date First Reported"
col_2_filter = "Date of vascular dementia report"
col_combined = "Vascular Dementia Report Date"

result_df = result_df.drop(columns=[col_1_filter, col_2_filter, col_combined])

# Save the result
result_df.to_csv('diagnosed.csv')

# Display the first 5 rows
result_df.describe()

,id,Birth Year,Sex,Education,BMI Impedance,Smoking Status,Ever Smoked,Alcohol Intake Frequency,Report of stroke,Diabetes Diagnosed By Doctor,...,Platelet distribution width,Red blood cell (erythrocyte) count,Red blood cell (erythrocyte) distribution width,Reticulocyte count,Reticulocyte percentage,White blood cell (leukocyte) count,Blood Pressure Diastolic,Blood Pressure Systolic,Pulse Rate at Blood Pressure,Has Vascular Dementia
count,2.186000e+03,2186.000000,2186.000000,2137.000000,2162.000000,2181.000000,2159.000000,2181.000000,681.000000,2181.000000,...,2037.000000,2037.000000,2037.000000,2005.000000,2005.000000,2037.000000,1983.000000,1983.000000,1983.000000,2186.0
mean,3.515518e+06,1943.111162,0.582342,-1.001404,28.557839,0.658414,0.670681,3.211371,9.027900,0.204493,...,16.542081,4.489278,13.725449,0.064314,1.435638,7.335621,81.993444,147.228946,70.938477,1.0
std,1.442844e+06,4.317173,0.493286,4.960943,5.139395,0.769999,0.470075,1.732522,6.190893,0.428779,...,0.531861,0.444468,1.094076,0.035324,0.804452,1.956365,11.199929,21.110899,13.162657,0.0
min,1.002012e+06,1936.000000,0.000000,-7.000000,15.757600,-3.000000,0.000000,-3.000000,0.000000,-3.000000,...,15.200000,1.720000,11.490000,0.012000,0.263000,2.370000,47.000000,78.000000,35.000000,1.0
25%,2.300567e+06,1940.000000,0.000000,-7.000000,25.062350,0.000000,0.000000,2.000000,0.000000,0.000000,...,16.200000,4.200000,13.100000,0.045000,1.020000,6.020000,74.000000,132.000000,62.000000,1.0
50%,3.511698e+06,1942.000000,1.000000,1.000000,27.794000,1.000000,1.000000,3.000000,11.000000,0.000000,...,16.500000,4.484000,13.530000,0.060000,1.330000,7.120000,82.000000,146.000000,70.000000,1.0
75%,4.741248e+06,1945.000000,1.000000,3.000000,31.421725,1.000000,1.000000,5.000000,11.000000,0.000000,...,16.860000,4.773000,14.100000,0.078000,1.740000,8.410000,89.000000,160.000000,79.000000,1.0
max,6.022126e+06,1968.000000,1.000000,6.000000,54.528300,2.000000,1.000000,6.000000,22.000000,1.000000,...,19.400000,6.390000,31.700000,1.077000,25.278000,34.130000,132.000000,241.000000,169.000000,1.0
